<a href="https://colab.research.google.com/github/VadbOss/Vadim-s_repa/blob/main/Algorithms_ML_030522.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.datasets import load_boston, load_wine

In [61]:
plt.figure(figsize=(10, 6))

<Figure size 720x432 with 0 Axes>

<Figure size 720x432 with 0 Axes>

In [ ]:
data = load_boston()

In [63]:
df = pd.DataFrame(columns=data.feature_names, data=data.data)
df['target'] = data.target

In [64]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  target   506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [66]:
X = df.drop(['target', 'CHAS'], axis=1)

In [ ]:
X[:5]

In [68]:
y = df['target']

In [69]:
import seaborn as sns

In [ ]:
sns.scatterplot(x='LSTAT', y= 'target', data=df)

In [71]:
#sns.pairplot(df)

In [ ]:
sns.scatterplot(x='RM', y='target', data=df)

In [73]:
df.CHAS.value_counts()

0.0    471
1.0     35
Name: CHAS, dtype: int64

In [ ]:
sns.regplot(x='RM', y='target', data=df)

In [ ]:
sns.regplot(x='LSTAT', y= 'target', data=df)

# Подключаем необходимые библиотеки

In [93]:
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn import preprocessing
import lightgbm
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [77]:
df.shape

(506, 14)

- Проверить адекватность колонок
- Обработать пропуски в данных
- Перевести категориальные данные в численные

In [ ]:
df.isna().sum()/df.shape[0]

Если есть пропуски - заменяем на:


1.  Моду
2.  Среднюю
3.  Nan
4.  Сообразно логике



In [79]:
# df['CRIM'].fillna(df['CRIM'].mode()[0], inplace=True)
# тоже самое можно сделать при помощи SimpleImputer из Sklearn

In [80]:
# Находим категориальные колонки и используем LabelEncoder для перевода в численные значения
cat_columns = [cname for cname in df.columns if df[cname].dtype == 'object']
encoder = preprocessing.LabelEncoder()

In [81]:
for col in cat_columns:
  df[col] = encoder.fit_transform(df[col])

Разделяем наши данные на тренировочные и проверочные датасеты (уже сделали выше)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [83]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(404, 12)
(102, 12)
(404,)
(102,)


In [ ]:
y_test

# Обучение BaseLine моделей

In [89]:
import numpy as np

In [96]:
# Линейная регрессия
lr = LinearRegression()
scores = cross_val_score(lr, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print('LinearRegression cross validation MAE: ', -np.mean(scores))   # добавляем минус спереди

LinearRegression cross validation MAE:  3.4404432633253457


In [97]:
# Дерево решений
dt = DecisionTreeRegressor()
scores = cross_val_score(dt, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print('DecisionTreeRegressor cross validation MAE: ', -np.mean(scores))   # добавляем минус спереди

DecisionTreeRegressor cross validation MAE:  3.560617283950618


In [98]:
# Случайный лес
rf = RandomForestRegressor(random_state=42)
scores = cross_val_score(rf, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print('RandomForestRegressor cross validation MAE: ', -np.mean(scores))   # добавляем минус спереди

RandomForestRegressor cross validation MAE:  2.4077541975308647


Градиентный бустинг, работает как и RF, но в отличие от него обучается на ошибках предыдущих деревьев
то есть минимизируем ошибку, а не строим много деревьев

In [99]:
# LightGBM
lgb = lightgbm.LGBMRegressor(random_state=42)
scores = cross_val_score(lgb, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print('Cross validation MAE: ', -np.mean(scores))   # добавляем минус спереди

Cross validation MAE:  2.444357115527016


# Подбор параметров и окончательный выбор модели

In [105]:
def cv_params(model, param_grid):
  scoring = 'neg_mean_absolute_error'

  opt_params = GridSearchCV(
      estimator= model,                  # Модель
      param_grid= param_grid,            # Параметры
      scoring= scoring,                  # Стратегия валидации
      cv= 5,                             # Количество слоев кросс-валидации
      n_jobs= -1)                        # Количество потоков для обучения, -1 = все

  opt_params.fit(X_train, y_train)
  params = opt_params.best_params_
  best_score = opt_params.best_score_

  print(f"Best score: {round(-best_score, 2)}")
  print(f"Best parameters: {params}")

  return params

In [113]:
lgb_param_grid = {
    'max_depth': [4, 10, 15, -1],                 # Максимальная глубина дерева
    'num_leaves': [25, 35, 45],                   # Максимальное количество листьев на деревьях
    'n_estimators': [41, 100, 250, 500, 600]      # Количество деревьев
}

lgb_clean = lightgbm.LGBMRegressor(random_state=1)
lgb_params = cv_params(lgb_clean, lgb_param_grid)

Best score: 2.4
Best parameters: {'max_depth': 4, 'n_estimators': 250, 'num_leaves': 25}


In [115]:
rf_param_grid = {
    'max_depth': [20, 25],
    'n_estimators': [500, 800]
}

rf_clean = RandomForestRegressor(random_state=1)
rf_params = cv_params(rf_clean, rf_param_grid)

Best score: 2.4
Best parameters: {'max_depth': 25, 'n_estimators': 500}


# Итоговая модель - LightGBM

Обучаем на лучших параметрах, смотрим MAPE

In [137]:
lgb = lightgbm.LGBMRegressor(**lgb_params)
lgb.fit(X_train, y_train)

preds = lgb.predict(X_test)

print(f"MAPE: {round(mean_absolute_percentage_error(y_test, preds)*100, 2)}%")
print(f"MAE: {round(mean_absolute_error(y_test, preds), 2)}")

MAPE: 10.0%
MAE: 1.92


In [ ]:
results = pd.DataFrame({'Model': np.round(preds, 1), 'Actual': y_test})
results.reset_index()   #.drop('index', axis=1)

In [131]:
results.reset_index().drop('index', axis=1)

,Model,Actual
0,25.9,23.6
1,32.2,32.4
2,14.0,13.6
3,23.6,22.8
4,16.4,16.1
...,...,...
97,10.5,17.9
98,11.8,9.6
99,17.1,17.2
100,19.3,22.5


# Далее обучаем модель на ВСЕХ данных!!!

In [138]:
lgb = lightgbm.LGBMRegressor(**lgb_params)
lgb.fit(X, y)

preds2 = lgb.predict(X_test)

print(f"MAPE: {round(mean_absolute_percentage_error(y_test, preds2)*100, 2)}%")
print(f"MAE: {round(mean_absolute_error(y_test, preds2), 2)}")

MAPE: 4.32%
MAE: 0.8


In [139]:
results2 = pd.DataFrame({'Model': np.round(preds2, 1), 'Actual': y_test})
results2.reset_index()   #.drop('index', axis=1)

,index,Model,Actual
0,173,25.5,23.6
1,274,32.5,32.4
2,491,14.2,13.6
3,72,22.6,22.8
4,452,16.6,16.1
...,...,...,...
97,412,14.3,17.9
98,436,10.5,9.6
99,411,18.0,17.2
100,86,21.5,22.5
